In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%load_ext google.colab.data_table

Authenticated


Funkcja RAND() generuje pseudolosową wartość typu FLOAT64 z zakresu [0, 1), włącznie z 0 i bez 1. Sposób użycia jej do próbkowania jest podobny do tego, w jaki użyłbyś funkcji FARM_FINGERPRINT, ale nie musisz określać żadnego istniejącego klucza. RAND() zapewnia równomierny rozkład, więc jeśli niektóre kolumny mają pochylenie, w próbce oczekuje się tego samego pochylenia. Przykład próbkowania 10% danych w tabeli:

In [ ]:
%%bigquery --project empik-ga360

SELECT * FROM Table WHERE RAND() < 0.1

SEM slownik

In [ ]:
%%bigquery --project empik-ga360

select campaign, campaignId 

from (
    
SELECT
    date,
    trafficSource.campaign, # Wartość kampanii. Zwykle wyznaczana przez parametr adresu URL utm_campaign
    totals.visits, # Liczba sesji (dla wygody). Ta wartość wynosi 1 dla sesji ze zdarzeniami interakcji; zero – jeśli w sesji nie ma żadnych zdarzeń interakcji.
    trafficSource.adwordsClickInfo.campaignId, # Ta sekcja zawiera informacje o kliknięciach w Google Ads, jeśli są jakieś kliknięcia powiązane z daną sesją. Analytics korzysta z modelu ostatniego kliknięcia niebezpośredniego.
    RANK() OVER(PARTITION BY trafficSource.adwordsClickInfo.campaignId ORDER BY date DESC, totals.visits desc, rand() desc) AS rank,
FROM
    `empik-ga360.304756.ga_sessions_*`
WHERE
    -- _table_suffix between '20211226' and '20211226'
    _table_suffix = FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
    and trafficSource.adwordsClickInfo.campaignId is not null
group by 1,2,3,4
)
where rank = 1
limit 5